# Bonus: Temperature Analysis I

In [1]:
import pandas as pd
from datetime import datetime as dt
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import json

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:

# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement

Station = Base.classes.station

# Create our session (link) from Python to the DB
session = Session(engine)

In [4]:
# "tobs" is "temperature observations"
df = pd.read_csv("Resources/hawaii_measurements.csv")
df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [5]:
# Convert the date column format from string to datetime
df['date'] =pd.to_datetime(df['date'], format='%Y-%m-%d')

In [6]:
# Set the date column as the DataFrame index
df=df.set_index('date')
df.head()

,station,prcp,tobs
date,,,
2010-01-01,USC00519397,0.08,65
2010-01-02,USC00519397,0.00,63
2010-01-03,USC00519397,0.00,74
2010-01-04,USC00519397,0.00,76
2010-01-06,USC00519397,NaN,73


### Compare June and December data across all years 

In [7]:
# Import scipy for statistics
from scipy import stats
import scipy
from scipy import stats
from scipy.stats import ttest_ind

In [8]:
# Filter data for desired months
# Temperature in June

temp_june = session.query(Measurement.date, Measurement.tobs).\
                    filter(func.strftime("%m", Measurement.date) == '06').\
                    order_by(Measurement.date).all()

# Load data to a dataframe
temp_june_df = pd.DataFrame(temp_june, columns=['Date', 'Temperature'])

#Drop null values
temp_june=df[df.index.month==6].dropna()
# Sort dataframe by date
temp_june_df = temp_june_df.sort_values(by='Date', ascending=True)

temp_june_df.head()

,Date,Temperature
0,2010-06-01,78.0
1,2010-06-01,74.0
2,2010-06-01,73.0
3,2010-06-01,77.0
4,2010-06-01,69.0


In [9]:
# # Temperature in December
temp_dec = session.query(Measurement.date, Measurement.tobs).\
                    filter(func.strftime("%m", Measurement.date) == '12').\
                    order_by(Measurement.date).all()

# Load data to a dataframe
temp_dec_df = pd.DataFrame(temp_dec, columns=['Date', 'Temperature'])

# drop null values
temp_dec=df[df.index.month==12].dropna()
# Sort dataframe by date
temp_dec_df = temp_dec_df.sort_values(by='Date', ascending=True)
# Set date as index
temp_dec_df.set_index('Date', inplace=True)
temp_june_df.head()

,Date,Temperature
0,2010-06-01,78.0
1,2010-06-01,74.0
2,2010-06-01,73.0
3,2010-06-01,77.0
4,2010-06-01,69.0


In [24]:
# Identify the average temperatures
#For June
jun_temp_mean=round(temp_june_df['Temperature'].mean(),1)
print(f'The average temperateure for June is {jun_temp_mean}F.') 

#For December
dec_temp_mean=round(temp_dec_df['Temperature'].mean(),1)
print(f'The average temperature for December is {dec_temp_mean}F.')

The average temperateure for June is 74.9F.
The average temperature for December is 71.0F.


In [11]:
# Create collections of June temperature data
temp_june_df.describe()

,Temperature
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


In [12]:
# Create collections of December temperature data
temp_dec_df.describe()

,Temperature
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


In [32]:
# Run paired t-test

stats.ttest_ind(temp_june_df['Temperature'], temp_dec_df['Temperature'], equal_var = False)



Ttest_indResult(statistic=31.355036920962423, pvalue=4.193529835915755e-187)

In [37]:
alpha = 0.05
pvalue = 4.193529835915755e-187

# interpret via p-value
if pvalue > alpha:
    print('Retain null hypothesis that the means are equal.\
        \nThe difference in means is NOT statistically significant')
else:
    print('Reject the null hypothesis that the means are equal.\
          \nThe difference in means is statistically significant')

Reject the null hypothesis that the means are equal.          
The difference in means is statistically significant


### Analysis

*Based on the paired ttest results the pvalue is very small,

*The difference is statistically significant, 

*Ttherefore we reject the null hypothesis that the means of temperature in June are equal to the avg temperature in December, while also noting that temperature measurements may vary by stations or locations.  

### Close session

In [ ]:
# Close Session
session.close()